In [85]:
#Import Dependencies
import pandas as pd

In [86]:
state_data = 'publicAssistanceData/output_data/state_data/'
states = ["Alabama","Alaska","Arizona","Arkansas","California","Colorado",
  "Connecticut","Delaware","Florida","Georgia","Hawaii","Idaho","Illinois",
  "Indiana","Iowa","Kansas","Kentucky","Louisiana","Maine","Maryland",
  "Massachusetts","Michigan","Minnesota","Mississippi","Missouri","Montana",
  "Nebraska","Nevada","NewHampshire","NewJersey","NewMexico","NewYork",
  "NorthCarolina","NorthDakota","Ohio","Oklahoma","Oregon","Pennsylvania",
  "RhodeIsland","SouthCarolina","SouthDakota","Tennessee","Texas","Utah",
  "Vermont","Virginia","Washington","WestVirginia","Wisconsin","Wyoming", "DistrictofColumbia"]
dfs = []
for state in states:
    df = pd.read_csv(f'publicAssistanceData/output_data/state_data/{state}Public.csv')
    df['state'] = state
    dfs.append(df)
len(dfs)

51

In [87]:
#Extra cleaning of FEMA data for merging
def data_clean(df):
    #Remove statewide data
    
    #Changing date to year
    dates = df.declarationDate
    years = []
    for date in dates:
        year = int(date[0:4])
        years.append(year)
    df['declarationDate'] = years
    df = df.rename(columns={'declarationDate':'year','disasterNumber':'disasterID', 'amountOblig':'FEMA$'})
    #Remove data before '03
    df = df[df['year'] >= 2003]
    df = df[df['year'] <= 2018]
    return df

In [88]:
clean_dfs = []
for df in dfs:
    data_clean(df)
    clean_dfs.append(df)
len(clean_dfs)

51

In [89]:
#concat all states
total_df = clean_dfs[0]
for df in clean_dfs:
   total_df = pd.concat([total_df, df])

In [100]:
#group data by incident type then county
total_group = total_df.groupby(['incidentType', 'state', 'county'])
total_df2 = pd.DataFrame({'incidentCount':total_group['disasterNumber'].count(), 'FEMA$':total_group.amountOblig.mean()})
total_df2 = total_df2.drop('Statewide', level=2)
total_df2.tail()

incidentCount         FEMA$
incidentType state  county                              
Tsunami      Hawaii Maui                 7  2.939823e+04
             Oregon Coos                 3  4.454609e+04
                    Curry                5  1.018568e+06
                    Lincoln              3  6.313122e+04
Volcano      Hawaii Hawaii              14  9.136473e+05

In [101]:
#Export new df into csv
total_df2.to_csv('publicAssistanceData/byDisasterandCounty.csv')

In [105]:
#Read in DD summary
declare_data = pd.read_csv('disaster declaration sum2.csv')
declare_data = declare_data[declare_data['fyDeclared'] >= 2003]
declare_data = declare_data[declare_data['fyDeclared'] <= 2018]
declare_data.head()

,Unnamed: 0,disasterNumber,fipsStateCode,fyDeclared,declarationType,incidentType,fipsCountyCode,state county
13601,13601,1444,39,2003,DR,Severe Storm(s),49,"39,49"
13613,13613,1453,39,2003,DR,Severe Storm(s),141,"39,141"
14309,14309,1453,39,2003,DR,Severe Storm(s),97,"39,97"
14602,14602,1444,39,2003,DR,Severe Storm(s),77,"39,77"
14672,14672,1458,51,2003,DR,Severe Storm(s),840,"51,840"


In [106]:
#Clean decleration data
declare_data = declare_data.drop(['Unnamed: 0', 'fyDeclared', 'incidentType'], axis=1)
declare_data.head()

,disasterNumber,fipsStateCode,declarationType,fipsCountyCode,state county
13601,1444,39,DR,49,"39,49"
13613,1453,39,DR,141,"39,141"
14309,1453,39,DR,97,"39,97"
14602,1444,39,DR,77,"39,77"
14672,1458,51,DR,840,"51,840"
